# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Sexism Detection, Multi-class Classification, LLMs, Prompting


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Relevant Material

- Tutorial 3
- Huggingface documentation
- Huggingface hub

# Introduction

You are tasked to address the [EDOS Task A](https://github.com/rewire-online/edos) on sexism detection.

## Problem definition

Given an input text sentence, the task is to label the sentence as sexist or not sexist (binary classification).

### Examples:

**Text**: *``Schedule a date with her, then don't show up. Then text her "GOTCHA B___H".''*

**Label**: Sexist

**Text**: *``That’s completely ridiculous a woman flashing her boobs is not sexual assault in the slightest.''*

**Label**: Not sexist



## Approach

We will tackle the binary classification task with LLMs.

In particular, we'll consider zero-/few-shot prompting approaches to assess the capability of some popular open-source LLMs on this task.

## Preliminaries

We are going to download LLMs from [Huggingface](https://huggingface.co/).

Many of these open-source LLMs require you to accept their "Community License Agreement" to download them.

In summary:

- If not already, create an account of Huggingface (~2 mins)
- Check a LLM model card page (e.g., [Mistral v3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)) and accept its "Community License Agreement".
- Go to your account -> Settings -> Access Tokens -> Create new token -> "Repositories permissions" -> add the LLM model card you want to use.
- Save the token (we'll need it later)

### Huggingface Login

Once we have created an account and an access token, we need to login to Huggingface via code.

- Type your token and press Enter
- You can say No to Github linking

In [53]:
import os
from huggingface_hub import login

token = os.getenv("HUGGINGFACE_TOKEN")

login(token)

After login, you can download all models associated with your access token in addition to those that are not protected by an access token.

### Data Loading

Since we are only interested in prompting, we do not require a train dataset.

We have preparared a small test set version of EDOS in our dedicated [Github repository](https://github.com/lt-nlp-lab-unibo/nlp-course-material).

Check the ``Assignment 2/data`` folder.
It contains:

- ``a2_test.csv`` → a small test set of 300 samples.
- ``demonstrations.csv`` -> a batch of 1000 samples for few-shot prompting.

Both datasets contain a balanced number of sexist and not sexist samples.


## Imports

In [54]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch
import numpy as np

PATH = '../data/'

### Instructions

We require you to:

* **Download** the ``A2/data`` folder.
* **Encode** ``a2_test.csv`` into a ``pandas.DataFrame`` object.

In [55]:
#Encode a2_test.csv into a pandas dataframe

df = pd.read_csv(PATH + 'a2_test.csv')
df.head()

,rewire_id,text,label_sexist
0,sexism2022_english-17133,The boys must be gaming because there goes the...,not sexist
1,sexism2022_english-14197,Look at those eyes. Either someone unexpectedl...,sexist
2,sexism2022_english-3018,Old man mogs everyone in this sub,not sexist
3,sexism2022_english-5301,"Excellent, I was just looking at another post ...",not sexist
4,sexism2022_english-17796,So you run back to daddy whenever you need hel...,sexist


# [Task 1 - 0.5 points] Model setup

Once the test data has been loaded, we have to setup the model pipeline for inference.

In particular, we have to:
- Load the model weights from Huggingface
- Quantize the model to fit into a single-GPU limited hardware

## Which LLMs?

The pool of LLMs is ever increasing and it's impossible to keep track of all new entries.

We focus on popular open-source models.

- [Mistral v2](mistralai/Mistral-7B-Instruct-v0.2)
- [Mistral v3](mistralai/Mistral-7B-Instruct-v0.3)
- [Llama v3.1](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
- [Phi3-mini](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)

Other open-source models are more than welcome!

### Instructions

In order to get Task 1 points, we require you to:

* Pick 2 model cards from the provided list.
* For each model:
  - Define a separate section of your notebook for the model.
  - Setup a quantization configuration for the model.
  - Load the model via HuggingFace APIs.


### Notes

1. There's a popular library integrated with Huggingface's ``transformers`` to perform quantization.

2. Define two separate sections of your notebook to show that you have implemented the prompting pipeline for each selected model card.

# Phi-3.5 mini instruct

In [56]:
# Load model directly

model_card = "microsoft/Phi-3.5-mini-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_card, trust_remote_code=True)

In [57]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_card,
    return_dict=True,
    quantization_config=bnb_config,
    device_map = "auto",
    trust_remote_code=True,
)

model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_fe

We set the configuration parameters

In [58]:
generation_config = model.generation_config
generation_config.max_new_tokens = 2
#generation_config.eos_token_id = tokenizer.eos_token_id
#generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.do_sample = False
generation_config.temperature = None
generation_config.top_p = None

In [59]:
gpu_name = torch.cuda.get_device_name(0)
print(f"GPU: {gpu_name}")

total_memory = torch.cuda.get_device_properties(0).total_memory
print(f"Total GPU memory: {total_memory / (1024 ** 3):.2f} GB")

allocated_memory = torch.cuda.memory_allocated(0)
print(f"Allocated GPU memory: {allocated_memory / (1024 ** 3):.2f} GB")

GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Total GPU memory: 8.00 GB
Allocated GPU memory: 4.23 GB


# [Task 2 - 1.0 points] Prompt setup

Prompting requires an input pre-processing phase where we convert each input example into a specific instruction prompt.


## Prompt Template

Use the following prompt template to process input texts.

In [60]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        TEXT:
        {text}

        ANSWER:
        """
    }
]

We use Hugging Face custom handling function on the prompt to create our final template

In [61]:
prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are an annotator for sexism detection.<|end|>
<|user|>
Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        TEXT:
        {text}

        ANSWER:
        <|end|>
<|assistant|>



Now we test the prompt on a sample text

In [62]:
example_text = "Ahahahah I fucking hate women. They are so stupid."
formatted_example = prompt.format(text=example_text)
parsed_example = tokenizer(formatted_example, return_tensors='pt').to('cuda')
generated = model.generate(input_ids=parsed_example['input_ids'],
                           attention_mask=parsed_example['attention_mask'],
                           generation_config=generation_config,
                           do_sample=False)
output = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
print(output)

You are an annotator for sexism detection. Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        TEXT:
        Ahahahah I fucking hate women. They are so stupid.

        ANSWER:
         YES


### Instructions

In order to get Task 2 points, we require you to:

* Write a ``prepare_prompts`` function as the one reported below.

In [63]:
def prepare_prompts(texts, prompt_template, tokenizer):
    """
      This function format input text samples into instructions prompts.

      Inputs:
        texts: input texts to classify via prompting
        prompt_template: the prompt template provided in this assignment
        tokenizer: the transformers Tokenizer object instance associated with the chosen model card

      Outputs:
        input texts to classify in the form of instruction prompts
    """
    tokenized_prompts = []
    texts_len = len(texts)

    for i, text in enumerate(texts):
        full_prompt = prompt_template.format(text=text)
        tokenized_prompt = tokenizer(full_prompt, return_tensors='pt').to('cuda')
        tokenized_prompts.append(tokenized_prompt)
        if i % 50 == 0:
            print(f"Tokenized {i}/{texts_len} prompts")

    return tokenized_prompts

Test


In [64]:
texts = ["Hhaaha I fucking hate women.", "Hello, my name is John."]
tokenized_prompts = prepare_prompts(texts, prompt, tokenizer)
print(tokenized_prompts)

Tokenized 0/2 prompts
[{'input_ids': tensor([[32006,   887,   526,   385,  9732,  1061,   363,  7916,  1608, 15326,
         29889, 32007, 32010,  3575,  3414,   338,   304,   770,  1598,  1881,
          1426,   408,  6943,  7916,  1608,   470,   451, 29889,  2538,  2818,
           871, 22483,   470, 11698, 29889,    13,    13,  4706,   323, 12194,
         29901,    13,  4706,   379,  2350, 25613,   306,   285,  2707,   292,
         26277,  5866, 29889,    13,    13,  4706,   319,  3059, 29956,  1001,
         29901,    13,   308, 32007, 32001]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}, {'input_ids': tensor([[32006,   887,   526,   385,  9732,  1061,   363,  7916,  1608, 15326,
         29889, 32007, 32010,  3575,  3414,   338,   304,   770,  1598,

### Notes

1. You are free to modify the prompt format (**not its content**) as you like depending on your code implementation.

2. Note that the provided prompt has placeholders. You need to format the string to replace placeholders. Huggingface might have dedicated APIs for this.

# [Task 3 - 1.0 points] Inference

We are now ready to define the inference loop where we prompt the model with each pre-processed sample.

### Instructions

In order to get Task 3 points, we require you to:

* Write a ``generate_responses`` function as the one reported below.
* Write a ``process_response`` function as the one reported below.

In [65]:
def generate_responses(model, prompt_examples):
  """
    This function implements the inference loop for a LLM model.
    Given a set of examples, the model is tasked to generate a response.

    Inputs:
      model: LLM model instance for prompting
      prompt_examples: pre-processed text samples

    Outputs:
      generated responses
  """
  responses = []

  for prompt in prompt_examples:
    outputs = model.generate(**prompt, max_new_tokens=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

  return responses

The maximum number of tokens for the output has to be slightly higher than 2, since the model needs some spacing margin to generate the answer.  
Since we have only two possible answers, we set the `max_new_tokens` value to 4.

Test

In [66]:
responses = generate_responses(model, tokenized_prompts)
print(responses[0])

You are an annotator for sexism detection. Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        TEXT:
        Hhaaha I fucking hate women.

        ANSWER:
         YES


In [67]:
def process_response(response):
    """
      This function takes a textual response generated by the LLM
      and processes it to map the response to a binary label.

      Inputs:
        response: generated response from LLM

      Outputs:
        parsed binary response: return 1 if YES and 0 if NO
    """
    result = response.split("\n")[-1].strip().lower()
    if "yes" in result:
        return 1
    elif "no" in result:
        return 0
    else:
        return -1 # Invalid response, it will then casted to 0 for accuracy calculation. It is used for the fail rate calculation.

Test

In [68]:
process_response(responses[0])

1

We can now run the inference loop on the dataset

In [69]:
def run_inference(df, model, prompt, tokenizer):
    """
    This function runs inference on a dataset using the LLM model.

    Inputs:
        df: input dataset
        model: LLM model instance for prompting
        prompt: the instruction prompt template
        tokenizer: the transformers Tokenizer object instance associated with the chosen model card

    Outputs:
        predictions: binary predictions for the input dataset
    """
    texts = df['text']
    tokenized_prompts = prepare_prompts(texts, prompt, tokenizer)
    print("Tokenized prompts generated.")
    responses = generate_responses(model, tokenized_prompts)
    print("Responses generated.")
    predictions = [process_response(response) for response in responses]
    print("Predictions generated.")

    return predictions

In [70]:
#Run inference on the dataset
predictions = run_inference(df, model, prompt, tokenizer)

Tokenized 0/300 prompts
Tokenized 50/300 prompts
Tokenized 100/300 prompts
Tokenized 150/300 prompts
Tokenized 200/300 prompts
Tokenized 250/300 prompts
Tokenized prompts generated.
Responses generated.
Predictions generated.


In [71]:
df['predictions'] = predictions

## Notes

1. According to our tests, it should take you ~10 mins to perform full inference on 300 samples.

# [Task 4 - 0.5 points] Metrics

In order to evaluate selected LLMs, we need to compute performance metrics.

In particular, we are interested in computing **accuracy** since the provided data is balanced with respect to classification classes.

Moreover, we want to compute the ratio of failed responses generated by models. 

That is, how frequent the LLM fails to follow instructions and provides incorrect responses that do not address the classification task.

We denote this metric as **fail-ratio**.

In summary, we parse generated responses as follows:
- 1 if the model says YES
- 0 if the model says NO
- 0 if the model does not answer in either way

### Instructions

In order to get Task 4 points, we require you to:

* Write a ``compute_metrics`` function as the one reported below.
* Compute metrics for the two selected LLMs.

In [72]:
def compute_metrics(responses, y_true):
    """
      This function takes predicted and ground-truth labels and compute metrics.
      In particular, this function compute accuracy and fail-ratio metrics.
      This function internally invokes `process_response` to compute metrics.

      Inputs:
        responses: generated LLM responses
        y_true: ground-truth binary labels

      Outputs:
        dictionary containing desired metrics
    """
    accuracy = 0
    fail_ratio = 0

    for i, response in enumerate(responses):
        if response == y_true[i]:
            accuracy += 1
        elif response == -1 and y_true[i] == 0:
            accuracy += 1
        if response == -1:
            fail_ratio += 1
    
    accuracy /= len(y_true)
    fail_ratio /= len(y_true)

    return {
        "accuracy": round(accuracy, 4),
        "fail_ratio": round(fail_ratio, 4)
    }

In [73]:
y_true = df['label_sexist'].apply(lambda x: 1 if x == 'sexist' else 0).tolist()
metrics = compute_metrics(df['predictions'], y_true)
print(metrics)

{'accuracy': 0.59, 'fail_ratio': 0.0}


# [Task 5 - 1.0 points] Few-shot Inference

So far, we have tested models in a zero-shot fashion: we provide the input text to classify and instruct the model to generate a response.

We are now interested in performing few-shot prompting to see the impact of providing demonstration examples.

To do so, we slightly change the prompt template as follows.

In [74]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        EXAMPLES:
        {examples}

        TEXT:
        {text}

        ANSWER:
        """
    }
]

## Instructions

In order to get Task 5 points, we require you to:

- Load ``demonstrations.csv`` and encode it into a ``pandas.DataFrame`` object.
- Define a ``build_few_shot_demonstrations`` function as the one reported below.
- Perform few-shot inference as in Task 3.
- Compute metrics as in Task 4.

We load ``demonstrations.csv`` and encode it into a ``pandas.DataFrame`` object


In [75]:
demonstrations = pd.read_csv(PATH + 'demonstrations.csv')
demonstrations.head()

,rewire_id,text,label_sexist
0,sexism2022_english-10348,I would never have my kid go to a daycare with...,not sexist
1,sexism2022_english-7950,Democrat women are so butt fucking ugly.,sexist
2,sexism2022_english-7742,If you really wanted to twist the knife you’d ...,not sexist
3,sexism2022_english-16170,Those women are pro-rape. They didn’t care abo...,sexist
4,sexism2022_english-9857,Married a bitch... Is it the standard scenario...,sexist


We define a ``build_few_shot_demonstrations`` function as the one reported below.

In [76]:
def build_few_shot_demonstrations(demonstrations, num_per_class=2):
  """
    Inputs:
      demonstrations: the pandas.DataFrame object wrapping demonstrations.csv
      num_per_class: number of demonstrations per class
 
    Outputs:
      a list of textual demonstrations to inject into the prompt template.
  """
  
  examples = []
  sexist_demonstrations = demonstrations[demonstrations['label_sexist'] == 'sexist']['text'].sample(num_per_class)
  non_sexist_demonstrations = demonstrations[demonstrations['label_sexist'] == 'not sexist']['text'].sample(num_per_class)
  examples.extend(sexist_demonstrations)
  examples.extend(non_sexist_demonstrations)

  for i in range(len(examples)):
    examples[i] =  f"TEXT: {examples[i]}\nANSWER: YES" if i < num_per_class else f"TEXT: {examples[i]}\nANSWER: NO"

  return examples

In [77]:
num_per_class = 2

np.random.seed(42)
examples = build_few_shot_demonstrations(demonstrations, num_per_class)
formatted_examples = "\n".join(examples)
print(formatted_examples)

TEXT: nothing irks a left wing feminist worse than seeing a beautiful virtuous woman who is conservative they HATE her H-A-T-E so they try to suppress her next it will be illegal to be born beautiful they'll call it "Hate Birth"
ANSWER: YES
TEXT: Get #TRIGGERED, Hillaryfags, femme-fags, and feminazis: [URL] #MAGA #TRUMP #ELECTION2016 [URL]
ANSWER: YES
TEXT: Reposting this since all the mailboxes were full over the weekend. Please everybody call! #ReleaseTheMemo
ANSWER: NO
TEXT: she is mental......they are teaching whites to hate their own children....
ANSWER: NO


We perform few-shot inference as in Task 3.

In [78]:
def prepare_prompts_few_shots(texts, examples, prompt_template, tokenizer):
    """
      This function format input text samples into instructions prompts.

      Inputs:
        texts: input texts to classify via prompting
        examples: list of examples to inject into the prompt template
        prompt_template: the prompt template provided in this assignment
        tokenizer: the transformers Tokenizer object instance associated with the chosen model card

      Outputs:
        input texts to classify in the form of instruction prompts
    """
    tokenized_prompts = []
    texts_len = len(texts)
    formatted_examples = "\n".join(examples)

    for i, text in enumerate(texts):
        full_prompt = prompt_template.format(text=text, examples=formatted_examples)
        tokenized_prompt = tokenizer(full_prompt, return_tensors='pt').to('cuda')
        tokenized_prompts.append(tokenized_prompt)
        if i % 50 == 0:
            print(f"Tokenized {i}/{texts_len} prompts")

    return tokenized_prompts

In [79]:
def run_inference_few_shots(df, model, prompt, tokenizer, num_per_class):
    """
    This function runs inference on a dataset using the LLM model.

    Inputs:
        df: input dataset
        model: LLM model instance for prompting
        prompt: the instruction prompt template
        tokenizer: the transformers Tokenizer object instance associated with the chosen model card
        num_per_class: number of examples for each class to inject into the prompt template

    Outputs:
        predictions: binary predictions for the input dataset
    """
    texts = df['text']
    examples = build_few_shot_demonstrations(demonstrations, num_per_class)
    tokenized_prompts = prepare_prompts_few_shots(texts, examples, prompt, tokenizer)
    print("Tokenized prompts generated.")
    responses = generate_responses(model, tokenized_prompts)
    print("Responses generated.")
    predictions = [process_response(response) for response in responses]
    for i in range(len(predictions)):
        if predictions[i] == -1:
            print(responses[i])
    print("Predictions generated.")

    return predictions

We then run the inference on the dataset, starting with 2 examples per classes

In [80]:
prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
predictions = run_inference_few_shots(df, model, prompt, tokenizer, num_per_class)
df['predictions'] = predictions

Tokenized 0/300 prompts
Tokenized 50/300 prompts
Tokenized 100/300 prompts
Tokenized 150/300 prompts
Tokenized 200/300 prompts
Tokenized 250/300 prompts
Tokenized prompts generated.
Responses generated.
Predictions generated.


In [81]:
y_true = df['label_sexist'].apply(lambda x: 1 if x == 'sexist' else 0).tolist()
metrics = compute_metrics(df['predictions'], y_true)
print(metrics)
print(df.value_counts(predictions))

{'accuracy': 0.6467, 'fail_ratio': 0.0}
1    196
0    104
Name: count, dtype: int64


In [86]:
num_per_class = 4

np.random.seed(42)
examples = build_few_shot_demonstrations(demonstrations, num_per_class)

predictions = run_inference_few_shots(df, model, prompt, tokenizer, num_per_class)
df['predictions'] = predictions

y_true = df['label_sexist'].apply(lambda x: 1 if x == 'sexist' else 0).tolist()
metrics = compute_metrics(df['predictions'], y_true)
print(metrics)
print(df.value_counts(predictions))

Tokenized 0/300 prompts
Tokenized 50/300 prompts
Tokenized 100/300 prompts
Tokenized 150/300 prompts
Tokenized 200/300 prompts
Tokenized 250/300 prompts
Tokenized prompts generated.
Responses generated.
Predictions generated.
{'accuracy': 0.6767, 'fail_ratio': 0.0}
0    167
1    133
Name: count, dtype: int64


## Notes

1. You are free to pick any value for ``num_per_class``.

2. According to our tests, few-shot prompting increases inference time by some minutes (we experimented with ``num_per_class`` $\in [2, 4]$).

# [Task 6 - 1.0 points] Error Analysis

We are now interested in evaluating model responses and comparing their performance.

This analysis helps us in understanding

- Classification task performance gap: are the models good at this task?
- Generation quality: which kind of responses do models generate?
- Errors: which kind of mistakes do models do?

### Instructions

In order to get Task 6 points, we require you to:

* Compare classification performance of selected LLMs in a Table.
* Compute confusion matrices for selected LLMs.
* Briefly summarize your observations on generated responses.

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...

# FAQ

Please check this frequently asked questions before contacting us.

### Model cards

You can pick any open-source model card you like.

We recommend starting from those reported in this assignment.

### Implementation

Everything can be done via ``transformers`` APIs.

However, you are free to test frameworks, such as [LangChain](https://www.langchain.com/), [LlamaIndex](https://www.llamaindex.ai/) [LitParrot](https://github.com/awesome-software/lit-parrot), provided that you correctly address task instructions.

### Bonus Points

0.5 bonus points are arbitrarily assigned based on significant contributions such as:

- Outstanding error analysis
- Masterclass code organization
- Suitable extensions
- Evaluate A1 dataset and perform comparison

Note that bonus points are only assigned if all task points are attributed (i.e., 6/6).

### Prompt Template

Do not change the provided prompt template.

You are only allowed to change it in case of a possible extension.

### Optimizations

Any kind of code optimization (e.g., speedup model inference or reduce computational cost) is more than welcome!

# The End